# Explore and Model StatFox Matchup Data
`mlb_bet_notebooks/model_statfox_matchups.ipynb`
- Explore features
- Convert historical moneylines to break-even probabilities
- Model pre-computed features with RF and maybe PCA
- Compare model predictions to historical moneylines
    - Use break-even probabilities as alternative model and compare ROC
Jonathan Sims 2020-02-24

In [17]:
import sys
import os.path
from bs4 import BeautifulSoup
from urllib.request import urlopen
import boto3
import numpy as np
import pandas as pd 
import pickle
import codecs

In [26]:
file_o = '20200223.skr_statfox.2012.tsv.gz'
file_l = '20200223.skr_statfox.2012.log'
glyear = 'GL2012.CSV'

#### Get list of games

In [23]:
glhead = pd.read_csv('GLHEADER.CSV',header=None)
gms = pd.read_csv(glyear,header=0,names=list(glhead[0]))
gms = gms[['date','team_h','team_v','score_h','score_v']]

teams = pd.read_csv('TEAM_NAMES.CSV',header=0,index_col=['name1'],usecols=['name1','name3'])
teams = teams['name3'].to_dict()

gms['team_h'] = gms['team_h'].map(lambda x: teams[x.upper()])
gms['team_v'] = gms['team_v'].map(lambda x: teams[x.upper()])

#### Save dataframe to tsv

In [32]:
df_all.to_csv(file_o, sep='\t')

In [33]:
test = pd.read_csv(file_o, sep='\t')

In [34]:
test.shape

(2422, 462)